[View in Colaboratory](https://colab.research.google.com/github/iampatgrady/FeatureTools-Instacart-Demo/blob/master/Feature_Tools_Demo_Predicting_Basket_Orders.ipynb)

# Setting up Project

In [0]:
# install dependencies
!pip install dask
!pip install featuretools
!pip install -U -q PyDrive

from IPython.display import clear_output
clear_output()

In [10]:
# clean workspace
!rm -rf top_features
!rm -rf data
!rm -rf partitioned_data
!rm data.zip
!rm -rf __MACOSX
!ls

datalab


# FeatureTools

In [8]:
import featuretools as ft
import pandas as pd
import numpy as np
import os
from random import sample
from tqdm import tqdm
ft.__version__

'0.1.21'

## Download partitioned data, and kaggle data

In [0]:
# https://s3.amazonaws.com/instacart-datasets/instacart_online_grocery_shopping_2017_05_01.tar.gz


In [11]:
#TODO replace with bq connection

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# data.zip
file_id = '1-Xy-dO8-xuyrYGqCDk_AVQ3qXdry80S4'
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('data.zip')
!unzip -o data.zip
!rm -rf __MACOSX
clear_output()
!ls

data  datalab  data.zip


## Generate Data Partitions

In [0]:
def make_user_sample(orders, order_products, departments, products, user_ids, out_dir):
    orders_sample = orders[orders["user_id"].isin(user_ids)]

    orders_keep = orders_sample["order_id"].values
    order_products_sample = order_products[order_products["order_id"].isin(orders_keep)]

    try:
        os.mkdir(out_dir)
    except:
        pass
    order_products_sample.to_csv(os.path.join(out_dir, "order_products__prior.csv"), index=None)
    orders_sample.to_csv(os.path.join(out_dir, "orders.csv"), index=None)
    departments.to_csv(os.path.join(out_dir, "departments.csv"), index=None)
    products.to_csv(os.path.join(out_dir, "products.csv"), index=None)



def generateDataPartitions(data_dir,partition_dir,target_users,fullset=False, chunksize=1000):
    order_products = pd.concat([pd.read_csv(os.path.join(data_dir,"order_products__prior.csv")),
                                pd.read_csv(os.path.join(data_dir, "order_products__train.csv"))])
    orders = pd.read_csv(os.path.join(data_dir, "orders.csv"))
    departments = pd.read_csv(os.path.join(data_dir, "departments.csv"))
    products = pd.read_csv(os.path.join(data_dir, "products.csv"))

    import string
    printable = set(string.printable) # Clean the input of all product names
    products.product_name = products.product_name.apply( lambda pn: filter(lambda x: x in printable, pn) )
    
    users_unique = orders["user_id"].unique()
    if (fullset):  # Process all records if True
      target_users = len(users_unique)
    part_num = 0
    try:
        os.mkdir(partition_dir)
    except:
        pass
    for i in tqdm(range(0, target_users, chunksize)):
        users_keep = users_unique[i: i+chunksize]
        make_user_sample(orders, order_products, departments, products, users_keep, os.path.join(partition_dir, "part_%d" % part_num))
        part_num += 1

In [15]:
# target_users is number of users to generate partitions for to speed up demonstration  should process full data for best prections, flip to "fullset=True", takes ~10 mins  
generateDataPartitions("data", "partitioned_data", 10000, fullset=True, chunksize=2000)  

100%|██████████| 104/104 [05:51<00:00,  3.38s/it]


## Load Data

In [0]:
def load_entityset(data_dir):
  order_products = pd.read_csv(os.path.join(data_dir, "order_products__prior.csv"))
  orders = pd.read_csv(os.path.join(data_dir, "orders.csv"))
  departments = pd.read_csv(os.path.join(data_dir, "departments.csv"))
  products = pd.read_csv(os.path.join(data_dir, "products.csv"))

  order_products = order_products.merge(products).merge(departments)

  def add_time(df):
    df.reset_index(drop=True)
    df["order_time"] = np.nan
    days_since = df.columns.tolist().index("days_since_prior_order")
    hour_of_day = df.columns.tolist().index("order_hour_of_day")
    order_time = df.columns.tolist().index("order_time")

    df.iloc[0, order_time] = pd.Timestamp('Jan 1, 2018') +  pd.Timedelta(df.iloc[0, hour_of_day], "h")
    for i in xrange(1, df.shape[0]):
      df.iloc[i, order_time] = df.iloc[i-1, order_time] \
                                + pd.Timedelta(df.iloc[i, days_since], "d") \
                                + pd.Timedelta(df.iloc[i, hour_of_day], "h")

    return df

  orders = orders.groupby("user_id").apply(add_time)
  order_products = order_products.merge(orders[["order_id", "order_time"]])
  order_products["order_product_id"] = order_products["order_id"].astype(str) + "_" + order_products["add_to_cart_order"].astype(str)
  order_products.drop(["product_id", "department_id", "add_to_cart_order"], axis=1, inplace=True)
  es = ft.EntitySet("instacart")


  es.entity_from_dataframe(entity_id="order_products",
                           dataframe=order_products,
                           index="order_product_id",
                           variable_types={"aisle_id": ft.variable_types.Categorical, "reordered": ft.variable_types.Boolean},
                           time_index="order_time")

  es.entity_from_dataframe(entity_id="orders",
                           dataframe=orders,
                           index="order_id",
                           time_index="order_time")

  es.add_relationship(ft.Relationship(es["orders"]["order_id"], es["order_products"]["order_id"]))

  es.normalize_entity(base_entity_id="orders", new_entity_id="users", index="user_id")
  es.add_last_time_indexes()

  es["order_products"]["department"].interesting_values = order_products["department"].value_counts().head(5).index.values.tolist()
  es["order_products"]["product_name"].interesting_values = order_products["product_name"].value_counts().head(50).index.values.tolist()
  return es

In [17]:
es = load_entityset("partitioned_data/part_1/")
es

Entityset: instacart
  Entities:
    order_products [Rows: 312903, Columns: 7]
    orders [Rows: 32062, Columns: 8]
    users [Rows: 2000, Columns: 2]
  Relationships:
    order_products.order_id -> orders.order_id
    orders.user_id -> users.user_id

## Make Labels

In [0]:
def make_labels(es, training_window, cutoff_time, prediction_window):
  prediction_window_end = cutoff_time + prediction_window
  t_start = cutoff_time - training_window

  orders = es["orders"].df
  ops = es["order_products"].df

  training_data = ops[(ops["order_time"] <= cutoff_time) & (ops["order_time"] > t_start)]
  prediction_data = ops[(ops["order_time"] > cutoff_time) & (ops["order_time"] < prediction_window_end)]

  users_in_training = training_data.merge(orders)["user_id"].unique()

  valid_pred_data = prediction_data.merge(orders)
  valid_pred_data = valid_pred_data[valid_pred_data["user_id"].isin(users_in_training)]

  # list of products  
  empty_dict = dict((p, 0) for p in es["order_products"]["product_name"].interesting_values)
  def bought_products(df, d):
    for p in df.product_name.values:
      if p in d:
        d[p] = 1
    if len(d.values()) == 1:
      return d.values()[0]
    else:
      return d.values()

  labels = valid_pred_data.groupby("user_id").apply((lambda x: bought_products(x, empty_dict.copy()))).reset_index()
  labels["cutoff_time"] = cutoff_time
  #  rename and reorder
  labels.columns = ["user_id", "label", "time",]
  labels = labels[["user_id", "time", "label"]]

  return labels

In [19]:
label_times = make_labels(es=es,
                          training_window = ft.Timedelta("90 days"), 
                          cutoff_time = pd.Timestamp('April 15, 2018'),
                          prediction_window = ft.Timedelta("90 days"))

label_times.sample(5)

,user_id,time,label
881,3322,2018-04-15,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
890,3334,2018-04-15,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, ..."
175,2259,2018-04-15,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
217,2321,2018-04-15,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, ..."
863,3295,2018-04-15,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, ..."


## Automated Feature Engineering

In [20]:
feature_matrix, features = ft.dfs(target_entity="users", 
                                  cutoff_time=label_times,
                                  training_window=ft.Timedelta("90 days"), # same as above
                                  entityset=es,
                                  verbose=True)
# encode categorical values
fm_encoded, features_encoded = ft.encode_features(feature_matrix, features)

Built 137 features
Elapsed: 00:34 | Remaining: 00:00 | Progress: 100%|██████████| Calculated: 11/11 chunks


## Machine Learning

In [0]:
X = fm_encoded
X["label"] = label_times["label"].values
X = X.fillna(0)
y = X.pop("label")


from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler 
label_encoder = LabelEncoder()
onehot_encoder = OneHotEncoder(sparse=False)
x_scaler = StandardScaler()


integer_encoded = label_encoder.fit_transform(y)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
encoded_y = onehot_encoder.fit_transform(integer_encoded)

normal_X = x_scaler.fit_transform(X=X)

### Random forest

In [0]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=400, n_jobs=-1)

In [23]:
def feature_importances(model, features, n=10):
    importances = model.feature_importances_
    zipped = sorted(zip(features, importances), key=lambda x: -x[1])
    for i, f in enumerate(zipped[:n]):
        print "%d: Feature: %s, %.3f" % (i+1, f[0].get_name(), f[1])

    return [f[0] for f in zipped[:n]]

clf.fit(normal_X, encoded_y)
top_features = feature_importances(clf, features_encoded, n=25)

1: Feature: COUNT(order_products WHERE department = produce), 0.020
2: Feature: SKEW(orders.order_hour_of_day), 0.015
3: Feature: SKEW(orders.order_dow), 0.014
4: Feature: STD(orders.order_hour_of_day), 0.013
5: Feature: SKEW(orders.PERCENT_TRUE(order_products.reordered)), 0.013
6: Feature: STD(orders.PERCENT_TRUE(order_products.reordered)), 0.013
7: Feature: STD(orders.order_dow), 0.013
8: Feature: SKEW(orders.COUNT(order_products)), 0.013
9: Feature: MEAN(orders.order_hour_of_day), 0.013
10: Feature: STD(orders.NUM_UNIQUE(order_products.department)), 0.013
11: Feature: SKEW(orders.NUM_UNIQUE(order_products.aisle_id)), 0.013
12: Feature: SKEW(orders.NUM_UNIQUE(order_products.product_name)), 0.013
13: Feature: MEAN(orders.NUM_UNIQUE(order_products.product_name)), 0.013
14: Feature: MEAN(orders.order_dow), 0.013
15: Feature: PERCENT_TRUE(order_products.reordered), 0.013
16: Feature: SUM(orders.NUM_UNIQUE(order_products.product_name)), 0.013
17: Feature: COUNT(order_products), 0.013
18: 

In [0]:
ft.save_features(top_features, "top_features")

In [0]:
from collections import OrderedDict
y_products = OrderedDict()
lt = label_times.copy()
for i in range(len(es["order_products"]["product_name"].interesting_values)):
  label = es["order_products"]["product_name"].interesting_values[i]
  y_products[label] = lt.label.apply((lambda x: x[i]))

In [0]:
classifiers = OrderedDict()
for column in pd.DataFrame(y_products):
  y_loop = pd.DataFrame(y_products)[column]
  loop_clf = RandomForestClassifier(n_estimators=400, n_jobs=-1)
  loop_clf.fit(X.values, y_loop.values )
  classifiers[column] = loop_clf.predict(X.values)

In [0]:
y_df = pd.DataFrame(y_products)

In [28]:
y_df["user_id"] = X.reset_index()["user_id"]
y.sample(10)

user_id
3856    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
2716    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
3974    [0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, ...
3977    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
2437    [0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, ...
2542    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
3759    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
2736    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
2249    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
3328    [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, ...
Name: label, dtype: object

In [0]:
product_ids = []
for i in range(0, len(y_df)):
  d = y_df.iloc[i].drop("user_id").to_dict()
  product_id_list = []
  for product_id, buyTF in d.iteritems():
    if buyTF == 1:
      product_id_list.append(product_id)
  if len(product_id_list) > 0:
    product_ids.append(product_id_list)
  else:
    product_ids.append(["None"])
y_df["product_ids"] = product_ids
y_df = y_df.get(["user_id", "product_ids"])

## Generate Sample Submission File

In [43]:
for i,e in y_df.iterrows():
  print len(e.product_ids), e.user_id, e.product_ids

8 2001 ['Banana', 'Large Lemon', 'Sparkling Water Grapefruit', 'Limes', 'Asparagus', 'Organic Half & Half', 'Organic Blackberries', 'Organic Zucchini']
5 2002 ['Organic Red Onion', 'Organic Whole Milk', 'Raspberries', 'Honeycrisp Apple', 'Organic Cilantro']
1 2003 ['Raspberries']
1 2004 ['Honeycrisp Apple']
7 2007 ['Organic Raspberries', 'Organic Blueberries', 'Organic Whole Milk', 'Organic Strawberries', 'Organic Blackberries', 'Organic Large Extra Fancy Fuji Apple', 'Organic Cilantro']
1 2008 ['None']
1 2010 ['Honeycrisp Apple']
2 2011 ['Strawberries', 'Organic Italian Parsley Bunch']
14 2012 ['Organic Red Bell Pepper', 'Seedless Red Grapes', 'Yellow Onions', 'Organic Gala Apples', 'Red Peppers', 'Bag of Organic Bananas', 'Organic Hass Avocado', 'Organic Granny Smith Apple', 'Carrots', 'Organic Zucchini', 'Fresh Cauliflower', 'Organic Unsweetened Almond Milk', 'Organic Large Extra Fancy Fuji Apple', 'Organic Cilantro']
3 2013 ['Apple Honeycrisp Organic', 'Organic Half & Half', 'Organ

2 2661 ['Seedless Red Grapes', 'Organic Whole Milk']
11 2662 ['Banana', 'Organic Baby Spinach', 'Strawberries', 'Original Hummus', 'Organic Baby Arugula', 'Large Lemon', 'Sparkling Water Grapefruit', 'Honeycrisp Apple', 'Organic Cucumber', 'Fresh Cauliflower', 'Organic Cilantro']
11 2663 ['100% Whole Wheat Bread', 'Organic Avocado', 'Cucumber Kirby', 'Banana', 'Organic Whole Milk', 'Raspberries', 'Spring Water', 'Large Lemon', 'Organic Half & Half', 'Organic Whole String Cheese', 'Organic Zucchini']
11 2664 ['Cucumber Kirby', 'Banana', 'Seedless Red Grapes', 'Strawberries', 'Organic Strawberries', 'Bag of Organic Bananas', 'Organic Hass Avocado', 'Apple Honeycrisp Organic', 'Organic Fuji Apple', 'Organic Cucumber', 'Organic Zucchini']
1 2665 ['Strawberries']
4 2667 ['Banana', 'Organic Whole Milk', 'Strawberries', 'Organic Italian Parsley Bunch']
2 2668 ['Organic Yellow Onion', 'Strawberries']
1 2669 ['Organic Baby Spinach']
3 2671 ['Organic Whole Milk', 'Raspberries', 'Organic Italian 

3419 ['Organic Red Bell Pepper', 'Organic Baby Spinach', 'Organic Yellow Onion', 'Raspberries', 'Spring Water', 'Sparkling Water Grapefruit', 'Organic Blackberries', 'Organic Zucchini']
12 3420 ['Organic Small Bunch Celery', 'Organic Avocado', 'Banana', 'Organic Raspberries', 'Half & Half', 'Raspberries', 'Strawberries', 'Spring Water', 'Sparkling Water Grapefruit', 'Limes', 'Organic Blackberries', 'Organic Garlic']
10 3421 ['Organic Whole Milk', 'Half & Half', 'Spring Water', 'Sparkling Water Grapefruit', 'Organic Strawberries', 'Organic Hass Avocado', 'Organic Half & Half', 'Organic Blackberries', 'Organic Garlic', 'Organic Cilantro']
16 3422 ['Organic Small Bunch Celery', 'Seedless Red Grapes', 'Organic Baby Spinach', 'Strawberries', 'Red Peppers', 'Organic Baby Arugula', 'Large Lemon', 'Limes', 'Organic Strawberries', 'Organic Granny Smith Apple', 'Apple Honeycrisp Organic', 'Organic Italian Parsley Bunch', 'Organic Whole String Cheese', 'Honeycrisp Apple', 'Organic Large Extra Fan

## Scaling to Full Dataset

In [0]:
# ...tbd